In [1]:
import os

In [2]:
%pwd

'd:\\Dynamic-Price-Prediction-using-MLOps-and-DVC\\Dynamic-Price-Prediction-using-MLOps-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Dynamic-Price-Prediction-using-MLOps-and-DVC\\Dynamic-Price-Prediction-using-MLOps-and-DVC'

In [91]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [92]:
from dynamic_price.constants import *
from dynamic_price.utils.common import read_yaml, create_directories

In [93]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [94]:
import os
from dynamic_price import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

In [95]:
class DataTransformationConfig:
    def __init__(self, data_path, root_dir):
        self.data_path = data_path
        self.root_dir = root_dir

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def label_encode_columns(self, data, columns):
        label_encoder = preprocessing.LabelEncoder()

        for column in columns:
            data[column] = label_encoder.fit_transform(data[column])

        return data

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Label encode specific columns
        columns_to_encode = ['Location_Category', 'Time_of_Booking', 'Vehicle_Type', 'Customer_Loyalty_Status']
        data = self.label_encode_columns(data, columns_to_encode)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)


        print(train.shape)
        print(test.shape)
      

    
        

In [96]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-12-27 18:28:16,953: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-27 18:28:16,962: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-27 18:28:16,969: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-27 18:28:16,972: INFO: common: created directory at: artifacts]
[2023-12-27 18:28:16,981: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-27 18:28:17,194: INFO: 2788408301: Splited data into training and test sets]
[2023-12-27 18:28:17,194: INFO: 2788408301: (750, 10)]
[2023-12-27 18:28:17,197: INFO: 2788408301: (250, 10)]
(750, 10)
(250, 10)
